In [12]:
import pandas as pd
from dateutil import parser
# result_df = pd.read_csv("../../data/clean_data.100k.csv")
node_df = pd.read_parquet("../../data/node_semantic_df.parquet")

## 5. Добавление новой ноды + новой связи

- Подготовить фичи для новой ноды.
- Добавить новые связи.
- Сделать inference на уже обученной GraphSAGE.

1. `new_node_features` — это вектор, который должен включать:

- числовые агрегаты ноды (total_visits, avg_time, bounce_rate и т.д.)
- категориальные коды (category)
- TF-IDF embedding (screen + feature + action + category)
- все в том же порядке, что и graph.x

2. `edge_attr` — массив признаков для новой связи, размерность = graph.edge_attr.shape[1]. Если нет особых признаков — можно заполнить нулями.

3. 'NEW' — специальный маркер для новой ноды при указании src или dst.

4. **Предсказания:**
- `new_node_pred` — прогноз churn_rate для новой ноды
- `new_edge_preds` — прогноз потока пользователей по новым ребрам

In [61]:
# add_node_edge_inference.py
import torch
import numpy as np
from torch_geometric.data import Data

def add_new_node_and_edges(graph: Data,
                           model,
                           new_node_features: np.ndarray,
                           new_edges: list,
                           device='cpu'):
    """
    Добавляет новую ноду и новые связи в существующий граф и делает предсказания.

    Параметры:
    - graph: PyG Data объект, уже обученный
    - model: обученный GraphSAGE
    - new_node_features: 1D numpy array, длина = graph.x.shape[1]
    - new_edges: list of tuples (src_idx, dst_idx, edge_attr_array)
        - если новый узел участвует, можно использовать dst_idx='NEW' или src_idx='NEW'
    - device: 'cpu' или 'cuda'

    Возвращает:
    - new_node_pred: float, предсказанный churn_rate для новой ноды
    - new_edge_preds: list of float, предсказанные переходы по новым связям
    """

    model.eval()
    device = torch.device(device)
    graph = graph.to(device)
    model = model.to(device)

    # 1. Добавляем новую ноду
    x_new = torch.cat([graph.x.cpu(), torch.tensor(new_node_features, dtype=torch.float).unsqueeze(0)], dim=0)

    # 2. Добавляем новые связи
    edge_idx_list = [graph.edge_index.cpu().numpy()[0].tolist(),
                     graph.edge_index.cpu().numpy()[1].tolist()]
    edge_attr_list = graph.edge_attr.cpu().numpy().tolist() if graph.edge_attr is not None else []

    N = graph.num_nodes  # индекс новой ноды

    new_edge_positions = []
    for src, dst, edge_attr in new_edges:
        src_idx = N if src == 'NEW' else src
        dst_idx = N if dst == 'NEW' else dst
        edge_idx_list[0].append(src_idx)
        edge_idx_list[1].append(dst_idx)
        edge_attr_list.append(np.array(edge_attr).astype(float))
        new_edge_positions.append(len(edge_idx_list[0]) - 1)  # позиции новых ребер

    edge_index_new = torch.tensor(edge_idx_list, dtype=torch.long)
    edge_attr_new = torch.tensor(np.vstack(edge_attr_list), dtype=torch.float)

    data_new = Data(x=x_new, edge_index=edge_index_new, edge_attr=edge_attr_new).to(device)

    # 3. Forward pass
    with torch.no_grad():
        node_pred, edge_pred, _ = model(data_new.x, data_new.edge_index, data_new.edge_attr)



    print('new_edge_positions', edge_pred[new_edge_positions[0]])


    # 4. Результаты
    new_node_pred = node_pred[N].cpu().item()
    new_edge_preds = [edge_pred[i].cpu().item() for i in new_edge_positions]

    return new_node_pred, new_edge_preds


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: дублирующийся код
class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, num_layers=2, edge_dim=0):
        super().__init__()

        # Сохраняем параметры как атрибуты класса
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.edge_dim = edge_dim

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers-1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        # Node regression head
        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels//2),
            nn.ReLU(),
            nn.Linear(hidden_channels//2, 1)  # churn_rate scalar
        )

        # Edge regression head: we'll use concatenation of src_emb || dst_emb || edge_attr
        self.edge_dim = edge_dim
        edge_input_dim = hidden_channels * 2 + edge_dim
        self.edge_mlp = nn.Sequential(
            nn.Linear(edge_input_dim, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 1)  # transition_count (or normalized)
        )

    def forward(self, x, edge_index, edge_attr=None):
        # x: [N, in_channels], edge_index: [2, E], edge_attr: [E, edge_dim]
        h = x
        for conv in self.convs:
            h = conv(h, edge_index)
            h = F.relu(h)

        # node preds
        node_pred = self.node_mlp(h)  # [N, 1]

        # edge preds
        if edge_attr is not None:
            src_idx = edge_index[0]
            dst_idx = edge_index[1]
            src_h = h[src_idx]
            dst_h = h[dst_idx]
            edge_input = torch.cat([src_h, dst_h, edge_attr], dim=1)
            edge_pred = self.edge_mlp(edge_input)  # [E, 1]
        else:
            edge_pred = None

        return node_pred, edge_pred, h  # also return node embeddings


### Подготовка данных для предсказания
- Загрузить из файла модель
- Загрузить граф
- Загрузить векторизатор
- Мапинг категорий

In [4]:
def load_complete_model(filepath, device='cpu'):
    """
    Загружает модель и все связанные объекты
    """
    device = torch.device(device)
    
    # Явно указываем weights_only=False
    data = torch.load(filepath, map_location=device, weights_only=False)
    
    # Создаем модель
    model = GraphSAGENet(**data['model_params'])
    
    # Загружаем веса
    model.load_state_dict(data['model_state_dict'])
    model.to(device)
    model.eval()
    
    return model, data['scaler'], data['node_id_map']

# Загрузка
trained_model, scaler, node_id_map = load_complete_model('trained_graphsage.pth', device='cpu')

import pickle

# Загрузка всей структуры
def load_graph_data(filename):
    with open(filename, 'rb') as f:
        loaded_data = pickle.load(f)
    
    graph = loaded_data['graph']
    node_id_map = loaded_data['node_id_map']
    scaler = loaded_data['scaler']
    numeric_cols = loaded_data['numeric_cols']
    
    return graph, node_id_map, scaler, numeric_cols

# Загружаем граф
graph, loaded_node_id_map, loaded_scaler, loaded_numeric_cols = load_graph_data('../../data/graph_data.pkl')


In [7]:
import pickle

def load_vectorizer(filename='tfidf_vectorizer.pkl'):
    with open(filename, 'rb') as f:
        vectorizer = pickle.load(f)
    print(f"Vectorizer загружен из {filename}")
    print(f"Размер словаря: {len(vectorizer.vocabulary_)}")
    return vectorizer

vectorizer = load_vectorizer('../../data/tfidf_vectorizer.pkl')

def build_node_text_embedding(screen, feature, action, vectorizer):
    text = f"{screen} {feature} {action}"
    vec = vectorizer.transform([text]).toarray()
    return vec.squeeze()  # (64,)


Vectorizer загружен из ../../data/tfidf_vectorizer.pkl
Размер словаря: 64


TODO: `Обязательно сохранить соответствие категорий → кодов, иначе будет рассинхрон!`

In [8]:
def build_category_maps_from_graph(node_df_original):
    """
    node_df_original — тот DataFrame, ИЗ КОТОРОГО строился граф
    """
    cat_maps = {}
    cat_cols = [
        'screen',
        'feature',
        'action',
        'device_vendor_top',
        'device_type_top',
        'os_top'
    ]

    for col in cat_cols:
        cat = node_df_original[col].astype('category')
        cat_maps[col] = dict(zip(cat.cat.categories, range(len(cat.cat.categories))))

    return cat_maps

### Сборка фичей новой ноды

In [19]:
import numpy as np
import torch

def build_new_node_features(
    *,
    screen,
    feature,
    action,
    numeric_defaults,
    device_vendor_top,
    device_type_top,
    os_top,
    vectorizer,
    category_maps
):
    """
    numeric_defaults — dict с числовыми фичами
    """

    # -----------------------------
    # 1. Категориальные → codes
    # -----------------------------
    def encode(col, value):
        if value in category_maps[col]:
            return category_maps[col][value]
        else:
            # неизвестная категория → -1
            return -1

    cat_values = {
        'screen': encode('screen', screen),
        'feature': encode('feature', feature),
        'action': encode('action', action),
        'device_vendor_top': encode('device_vendor_top', device_vendor_top),
        'device_type_top': encode('device_type_top', device_type_top),
        'os_top': encode('os_top', os_top),
    }

    # -----------------------------
    # 2. Числовые фичи (20)
    # -----------------------------
    numeric_features = np.array([
        cat_values['screen'],                 # 0
        cat_values['feature'],                # 1
        cat_values['action'],                 # 2
        numeric_defaults.get('total_visits', 0),     # 3
        numeric_defaults.get('sessions_with_node', 0),  # 4
        numeric_defaults.get('avg_visits_per_session', 0),  # 5
        numeric_defaults.get('median_visits_per_session', 0),  # 6
        numeric_defaults.get('avg_session_length', 0),  # 7
        numeric_defaults.get('avg_time_on_page_seconds', 0),  # 8
        numeric_defaults.get('median_time_on_page_seconds', 0),  # 9
        numeric_defaults.get('avg_session_duration_seconds', 0),  # 10
        numeric_defaults.get('repeat_ratio', 0),      # 11
        numeric_defaults.get('churn_count', 0),       # 12
        numeric_defaults.get('bounce_count', 0),      # 13
        numeric_defaults.get('bounce_rate', 0),       # 14
        numeric_defaults.get('avg_age', 0),            # 15
        numeric_defaults.get('male_ratio', 0),         # 16
        cat_values['device_vendor_top'],        # 17
        cat_values['device_type_top'],          # 18
        cat_values['os_top'],                   # 19
    ], dtype=np.float32)

    assert numeric_features.shape[0] == 20

    # -----------------------------
    # 3. TF-IDF embedding (64)
    # -----------------------------
    text_emb = build_node_text_embedding(
        screen, feature, action, vectorizer
    ).astype(np.float32)

    assert text_emb.shape[0] == 64

    # -----------------------------
    # 4. Финальный вектор (84)
    # -----------------------------
    node_features = np.concatenate([numeric_features, text_emb])
    assert node_features.shape[0] == 84

    return torch.tensor(node_features, dtype=torch.float)


### Добавление новой ноды

In [10]:
def add_new_node_and_edges(
    graph,
    model,
    new_node_features,
    new_edges,
    device='cpu'
):
    """
    new_edges: list of (src, dst, edge_attr)
    src / dst — int (индексы)
    """

    model.eval()
    graph = graph.to(device)
    model = model.to(device)

    # -----------------------------
    # 1. Добавляем ноду
    # -----------------------------
    new_node_features = new_node_features.to(device).unsqueeze(0)
    x = torch.cat([graph.x, new_node_features], dim=0)

    new_node_idx = graph.num_nodes

    # -----------------------------
    # 2. Добавляем рёбра
    # -----------------------------
    edge_index = graph.edge_index.clone()
    edge_attr = graph.edge_attr.clone()

    new_edge_indices = []
    new_edge_attrs = []

    for src, dst, attr in new_edges:
        if src == 'NEW':
            src = new_node_idx
        if dst == 'NEW':
            dst = new_node_idx

        new_edge_indices.append([src, dst])
        new_edge_attrs.append(attr)

    new_edge_indices = torch.tensor(new_edge_indices, dtype=torch.long, device=device).t()
    new_edge_attrs = torch.tensor(new_edge_attrs, dtype=torch.float, device=device)

    edge_index = torch.cat([edge_index, new_edge_indices], dim=1)
    edge_attr = torch.cat([edge_attr, new_edge_attrs], dim=0)

    new_graph = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr
    ).to(device)

    # -----------------------------
    # 3. Предсказание
    # -----------------------------
    with torch.no_grad():
        out = model(new_graph.x, new_graph.edge_index)
        preds = out[0].squeeze()

    return preds[new_node_idx].item()


In [66]:
x_mean = graph.x.mean(dim=0).cpu().numpy()

numeric_defaults = {
    'total_visits': x_mean[3],
    'sessions_with_node': x_mean[4],
    'avg_visits_per_session': x_mean[5],
    'median_visits_per_session': x_mean[6],
    'avg_session_length': x_mean[7],
    'avg_time_on_page_seconds': x_mean[8],
    'median_time_on_page_seconds': x_mean[9],
    'avg_session_duration_seconds': x_mean[10],
    'repeat_ratio': x_mean[11],
    'churn_count': x_mean[12],
    'bounce_count': x_mean[13],
    'bounce_rate': x_mean[14],
    'avg_age': x_mean[15],
    'male_ratio': x_mean[16],
}

category_maps = build_category_maps_from_graph(node_df)

new_node_features = build_new_node_features(
    screen='Дополнительно',
    feature='Выйти из приложения',
    action="Тап на кнопку 'Выйти из приложения'",
    # screen='Важное',
    # feature='Обращение к администратору',
    # action="Тап на кнопку 'Обращение к администратору'",
    numeric_defaults=numeric_defaults,
    device_vendor_top='Apple',
    device_type_top='iPhone',
    os_top='iOS',
    vectorizer=vectorizer,
    category_maps=category_maps
)

edge_dim = graph.edge_attr.size(1)
zero_edge_attr = np.zeros(edge_dim, dtype=np.float32)

# pred = add_new_node_and_edges(
#     graph,
#     trained_model,
#     new_node_features,
#     new_edges=[
#         # (0, 'NEW', zero_edge_attr),
#         ('NEW', 1, zero_edge_attr),
#     ],
#     device='cpu'
# )

# print('Predicted churn_rate for new node:', pred)


In [67]:
# Допустим, у нас уже есть обученная модель `trained_model` и Data `graph`

# 1) Подготовка признаков новой ноды
# - TF-IDF embedding: text_embedding для новой ноды
# - numeric features: среднее соседних нод или глобальное среднее
# new_node_raw = {
#     "screen": "Важное",
#     "feature": "Обращение к администратору",
#     "action": "Тап на кнопку Обращение к администратору"
# }

# new_text = (
#     new_node_raw["screen"] + " " +
#     new_node_raw["feature"] + " " +
#     new_node_raw["action"]
# )

# new_node_features = graph.x.mean(dim=0).cpu().numpy()  # простой пример

# 2) Подготовка новых ребер
# - edge_attr должен совпадать по размерности с graph.edge_attr
# E_attr_dim = graph.edge_attr.size(1) if graph.edge_attr is not None else 0
# sample_edge_attr = np.zeros(E_attr_dim, dtype=float)
edge_attr_mean = graph.edge_attr.mean(dim=0).cpu().numpy()
sample_edge_attr = edge_attr_mean.copy()

# Допустим, хотим добавить связи: существующая нода 0 -> новая, новая -> нода 1
new_edges = [
    (20, 'NEW', sample_edge_attr),    # edge from node 0 -> NEW
    ('NEW', 1, sample_edge_attr)     # edge NEW -> node 1
]

# 3) Получаем предсказания
new_node_pred, new_edge_preds = add_new_node_and_edges(graph, trained_model,
                                                      new_node_features,
                                                      new_edges,
                                                      device='cpu')

print("Predicted churn_rate for new node:", new_node_pred)
print("Predicted flows for new edges:", new_edge_preds)


new_edge_positions tensor([4.6790])
Predicted churn_rate for new node: 0.025099746882915497
Predicted flows for new edges: [4.67903470993042, 4.699269771575928]


/tmp/ipykernel_8547/1178868645.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_new = torch.cat([graph.x.cpu(), torch.tensor(new_node_features, dtype=torch.float).unsqueeze(0)], dim=0)
